# Imports

In [1]:
import requests
import pandas as pd
from datetime import datetime
import json
import sqlite3

# Função Lambda para realizar requisições

In [62]:
def lambda_handler(api_key, cidade):
    link = f'https://api.openweathermap.org/data/2.5/weather?q={cidade}&appid={api_key}'
    
    # Requisição
    requisicao = requests.get(link)
    
    # Retornando json de tempo
    return {
        'statusCode': requests.get(link).json()['cod'],
        'body': json.dumps(requisicao.json())
    }

# Parâmetros

In [30]:
# Credenciais
api_key = '1a0925695880013271c26100e80f7e3d'
cidade = 'santo andre'

# Realizando requisição e armazenando em variável

In [103]:
weather = json.loads(lambda_handler(api_key, cidade)['body'])

# Definindo informações que serão salvas em tabelas & abrindo conexão com o BD

In [107]:
infos = ['coord',
        'weather',
        'main',
        'wind',
        'clouds',
        'sys'
        ]

# Conexão com o BD
conexao = sqlite3.connect('itau.db')

In [108]:
general_infos = pd.DataFrame([[weather['base'],
            weather['visibility'],
            weather['dt'],
            weather['timezone'],
            weather['id'],
            weather['name'],
            weather['cod']]], columns=['base', 'visibility', 'dt', 'timezone', 'id', 'name', 'cod'])

# Salvando no BD tabelas com informações geográficas e climáticas

In [109]:
for i in infos:
    print(i)
    try:
        df = pd.DataFrame.from_dict(weather[i]) 
    except:
        df = pd.DataFrame.from_dict(weather[i], orient='index', columns=[i])
    df.to_sql(i + '_data_SA_api', conexao, if_exists='replace')
    print('ok')

general_infos.to_sql('general_infos_data_SA_api', conexao, if_exists='replace')

coord
ok
weather
ok
main
ok
wind
ok
clouds
ok
sys
ok


1

# Queries no BD para resgatar informação salva

In [110]:
for i in infos:
    rows = conexao.execute('select * from ' + i + '_data_SA_api')
    columns = [col[0] for col in rows.description]
    info = pd.DataFrame(rows.fetchall(), columns=columns)
    display(info)

,index,coord
0,lon,-46.5383
1,lat,-23.6639


,index,id,main,description,icon
0,0,800,Clear,clear sky,01d


,index,main
0,temp,294.46
1,feels_like,294.21
2,temp_min,290.06
3,temp_max,295.18
4,pressure,1026.00
5,humidity,60.00


,index,wind
0,speed,3.09
1,deg,110.00


,index,clouds
0,all,0


,index,sys
0,type,2
1,id,2033898
2,country,BR
3,sunrise,1690882887
4,sunset,1690922605


In [111]:
# Fechando conexão com o banco de dados
conexao.close()